Catena di Markov
 Cosa calcola?
Una probabilità, per ogni giocatore, che segni almeno 1 gol nella prossima giornata, a partire dallo stato attuale (ad es. “oggi ha fatto 1 gol”).


Procedura per iniziare (sottofase 2.3.1 – Caricamento e preparazione)

In [1]:
# STEP 1 – Caricamento del file da locale su Colab
from google.colab import files
uploaded = files.upload()  # Seleziona GolGiocatoriTemporale.csv dal tuo computer

# STEP 2 – Lettura del file CSV in un DataFrame
import pandas as pd

df = pd.read_csv('GolGiocatoriTemporale.csv')

# STEP 3 – Verifica del contenuto e visualizzazione delle prime righe
df.info()
df.head()


Saving GolGiocatoriTemporale.csv to GolGiocatoriTemporale (1).csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 39 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Giocatore  278 non-null    object
 1   G1         278 non-null    int64 
 2   G2         278 non-null    int64 
 3   G3         278 non-null    int64 
 4   G4         278 non-null    int64 
 5   G5         278 non-null    int64 
 6   G6         278 non-null    int64 
 7   G7         278 non-null    int64 
 8   G8         278 non-null    int64 
 9   G9         278 non-null    int64 
 10  G10        278 non-null    int64 
 11  G11        278 non-null    int64 
 12  G12        278 non-null    int64 
 13  G13        278 non-null    int64 
 14  G14        278 non-null    int64 
 15  G15        278 non-null    int64 
 16  G16        278 non-null    int64 
 17  G17        278 non-null    int64 
 18  G18        278 non-null    int64 
 19  G19  

,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,...,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
0,federico gatti,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,alessandro bastoni,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,andrea carboni,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tammy abraham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,francesco acerbi,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


##  Catene di Markov
Obiettivo:
Per ogni giocatore, stimare la probabilità che segni almeno 1 gol nella giornata 39 in base al suo stato attuale (giornata 38) e alle transizioni osservate nel suo storico (G1–G37).

Passo 1 – Definizione degli stati
Per semplicità e interpretabilità, definiamo 3 stati:

| Stato | Significato | Codice |
| ----- | ----------- | ------ |
| `0`   | Nessun gol  | `0`    |
| `1`   | 1 gol       | `1`    |
| `2`   | 2 o più gol | `2`    |

Questo codice:

*   rileva automaticamente quante giornate ci sono (n)
*   usa G1 fino a G(n-1) per costruire le transizioni
*   osserva Gn come stato corrente
*   stima la probabilità di segnare in G(n+1)

In [2]:
import pandas as pd
import numpy as np

# Caricamento del file CSV
df = pd.read_csv('GolGiocatoriTemporale.csv')

# Estrai dinamicamente le colonne G1 ... Gn
giornate = [col for col in df.columns if col.startswith('G') and col[1:].isdigit()]
nome_colonna_ultima = giornate[-1]
print(f" Ultima giornata trovata: {nome_colonna_ultima}")
print(f" Colonne giornate trovate: {giornate}")

nomi = df['Giocatore']
dati = df[giornate].copy()

# Funzione per trasformare i gol in stati
def gol_to_stato(x):
    if x == 0:
        return 0
    elif x == 1:
        return 1
    else:
        return 2

dati_stati = dati.map(gol_to_stato)

# Calcolo della matrice di transizione + probabilità per Gn+1
prob_previste = []

for idx, row in dati_stati.iterrows():
    transizioni = np.zeros((3, 3))  # 3 stati: 0,1,2

    for i in range(len(row) - 1):  # da G1 a G(n-1)
        stato_corrente = row.iloc[i]
        stato_successivo = row.iloc[i + 1]
        transizioni[stato_corrente][stato_successivo] += 1

    # Matrice di probabilità
    somma_per_riga = transizioni.sum(axis=1, keepdims=True)
    with np.errstate(divide='ignore', invalid='ignore'):
        matrice_prob = np.divide(transizioni, somma_per_riga, where=somma_per_riga != 0)

    # Stato attuale = ultima giornata
    stato_attuale = row.iloc[-1]
    prob = matrice_prob[stato_attuale][1] + matrice_prob[stato_attuale][2]
    prob_previste.append(prob)

# Colonna finale dinamica
col_markov = f'Prob_Gol_{len(giornate)+1}'
print(f"Colonna creata: {col_markov}")

# Crea DataFrame con i risultati
df_markov = pd.DataFrame({
    'Giocatore': nomi,
    col_markov: prob_previste
})

# Stampa tutte le righe
with pd.option_context('display.max_rows', None):
    print(df_markov)


 Ultima giornata trovata: G38
 Colonne giornate trovate: ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G27', 'G28', 'G29', 'G30', 'G31', 'G32', 'G33', 'G34', 'G35', 'G36', 'G37', 'G38']
Colonna creata: Prob_Gol_39
                   Giocatore  Prob_Gol_39
0             federico gatti     0.090909
1         alessandro bastoni     0.027778
2             andrea carboni     0.027778
3              tammy abraham     0.027778
4           francesco acerbi     0.088235
5                yacine adli     0.027778
6                alex sandro     0.000000
7            pontus almqvist     0.028571
8              amir rrahmani     0.058824
9              houssem aouar     0.000000
10          marko arnautovic     0.000000
11                    arthur     0.000000
12          kristjan asllani     0.027778
13           tommaso augello     0.027778
14             sardar azmou

# Controlli diagnostici post-Markov

Questo blocco di codice ha l’obiettivo di identificare eventuali anomalie o segnali sospetti nelle probabilità calcolate con il modello a Catene di Markov, in particolare i seguenti casi:

 Tipologie di controllo effettuate:

*  Probabilità esattamente pari a 1.0
 Segnale sospetto di overfitting: il modello ha visto sempre lo stesso comportamento e si aspetta con assoluta certezza che si ripeta. Nella realtà, un evento con probabilità 100% è molto raro, specialmente in contesti aleatori come il calcio.

* Probabilità esattamente pari a 0.0
 Anche questo può segnalare una stima anomala, ma più plausibile rispetto al caso precedente. Potrebbe dipendere da una mancanza di gol recenti oppure da transizioni storiche troppo rigide.

* Giocatori con un solo gol, proprio nell’ultima giornata disponibile
  Questi giocatori non avevano mai segnato prima, e hanno fatto il loro primo (e unico) gol nell’ultima giornata. È una situazione borderline, poiché il modello ha solo un dato “positivo” su cui basarsi e può sovrastimare l’impatto.

* Intersezione tra 1 e 3
 Questo è il caso più problematico: il giocatore ha segnato solo una volta (nell’ultima giornata) e il modello prevede con certezza (1.0) che segnerà di nuovo. È un chiaro sintomo di errore strutturale dovuto alla scarsità dei dati storici → in fase di calcolo della forma prevista, tale valore dovrà essere correttamente attenuato (ad esempio, introducendo un peso correttivo o un flag di attendibilità).

Nota: Il controllo su giocatori che non hanno mai segnato è stato omesso, poiché nel dataset GolGiocatoriTemporale.csv vengono inclusi solo giocatori che hanno effettuato almeno un gol (o autogol). La gestione del caso “mai segnato” verrà affrontata successivamente nella sottofase di calcolo della Forma_Prevista, eventualmente dando un peso o una penalizzazione specifica.

In [3]:
#  Nome colonna con probabilità stimata da Markov
col_markov = f'Prob_Gol_{len(giornate)+1}'

# Giocatori con probabilità esattamente 1.0
one_players = df_markov[df_markov[col_markov] == 1.0]['Giocatore'].tolist()

# Giocatori con probabilità esattamente 0.0
zero_players = df_markov[df_markov[col_markov] == 0.0]['Giocatore'].tolist()

# Giocatori che hanno segnato il loro unico gol proprio all'ultima giornata
cols_preultimo = [c for c in giornate if c != giornate[-1]]
solo_g_finale = df[
    (df[cols_preultimo].sum(axis=1) == 0) &
    (df[giornate[-1]] > 0)
]['Giocatore'].tolist()

#   Incrocio tra (1.0) e gol solo all’ultima giornata → possibile anomalia
anomalie = set(one_players).intersection(solo_g_finale)

#  Stampa dei risultati
print("Giocatori con probabilità esattamente 1.0:", one_players)
print("Giocatori con probabilità esattamente 0.0:", zero_players)
print(f"Giocatori che hanno segnato solo in {giornate[-1]}:", solo_g_finale)
print("Giocatori anomali (1.0 e unico gol nell'ultima giornata):", list(anomalie))


Giocatori con probabilità esattamente 1.0: ['junior sambia', 'mattia viti']
Giocatori con probabilità esattamente 0.0: ['alex sandro', 'houssem aouar', 'marko arnautovic', 'arthur', 'davide biraschi', 'matteo cancellieri', 'keinan davis', 'alessandro deiola', 'federico di francesco', 'kingstone mutandwa', 'tijjani noslin', 'mario pasalic', 'simy', 'vitor oliveira', 'mattia zaccagni']
Giocatori che hanno segnato solo in G38: ['alex sandro', 'keinan davis', 'alessandro deiola', 'kingstone mutandwa', 'junior sambia', 'mattia viti']
Giocatori anomali (1.0 e unico gol nell'ultima giornata): ['mattia viti', 'junior sambia']


In [4]:
import pandas as pd
import numpy as np

# Caricamento file
df = pd.read_csv('GolGiocatoriTemporale.csv')

# Estrai dinamicamente le colonne G1 ... Gn
giornate = [col for col in df.columns if col.startswith('G') and col[1:].isdigit()]
nome_colonna_ultima = giornate[-1]
print(f"Ultima giornata trovata: {nome_colonna_ultima}")
print(f"Colonne giornate trovate: {giornate}")

nomi = df['Giocatore']
dati = df[giornate].copy()

# Funzione: da gol a stato discreto
def gol_to_stato(x):
    if x == 0:
        return 0
    elif x == 1:
        return 1
    else:
        return 2

dati_stati = dati.map(gol_to_stato)

# Calcolo probabilità previsione con Laplace smoothing
prob_previste = []

for idx, row in dati_stati.iterrows():
    transizioni = np.ones((3, 3))  # Laplace smoothing: inizializza con 1 in ogni cella

    for i in range(len(row) - 1):  # da G1 a G(n-1)
        stato_corrente = row.iloc[i]
        stato_successivo = row.iloc[i + 1]
        transizioni[stato_corrente][stato_successivo] += 1

    # Calcolo matrice di probabilità
    somma_per_riga = transizioni.sum(axis=1, keepdims=True)
    matrice_prob = transizioni / somma_per_riga

    # Stato attuale (Gn)
    stato_attuale = row.iloc[-1]
    prob = matrice_prob[stato_attuale][1] + matrice_prob[stato_attuale][2]  # P(segna)

    prob_previste.append(prob)

# Nome colonna dinamico (es. Prob_Gol_39_Markov)
col_markov = f'Prob_Gol_{len(giornate)+1}'

df_markov = pd.DataFrame({
    'Giocatore': nomi,
    col_markov: prob_previste
})

# Visualizza le prime righe
df_markov.head()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_markov)


Ultima giornata trovata: G38
Colonne giornate trovate: ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G27', 'G28', 'G29', 'G30', 'G31', 'G32', 'G33', 'G34', 'G35', 'G36', 'G37', 'G38']


,Giocatore,Prob_Gol_39
0,federico gatti,0.138889
1,alessandro bastoni,0.076923
2,andrea carboni,0.076923
3,tammy abraham,0.076923
4,francesco acerbi,0.135135
5,yacine adli,0.076923
6,alex sandro,0.666667
7,pontus almqvist,0.078947
8,amir rrahmani,0.108108
9,houssem aouar,0.333333


In [5]:
#Nome colonna con probabilità stimata da Markov
col_markov = f'Prob_Gol_{len(giornate)+1}'

#Giocatori con probabilità esattamente 1.0
one_players = df_markov[df_markov[col_markov] == 1.0]['Giocatore'].tolist()

#Giocatori con probabilità esattamente 0.0
zero_players = df_markov[df_markov[col_markov] == 0.0]['Giocatore'].tolist()

#Giocatori che hanno segnato il loro unico gol proprio all'ultima giornata
cols_preultimo = [c for c in giornate if c != giornate[-1]]
solo_g_finale = df[
    (df[cols_preultimo].sum(axis=1) == 0) &
    (df[giornate[-1]] > 0)
]['Giocatore'].tolist()

# Incrocio tra (1.0) e gol solo all’ultima giornata → possibile anomalia
anomalie = set(one_players).intersection(solo_g_finale)

# Stampa dei risultati
print("Giocatori con probabilità esattamente 1.0:", one_players)
print("Giocatori con probabilità esattamente 0.0:", zero_players)
print(f"Giocatori che hanno segnato solo in {giornate[-1]}:", solo_g_finale)
print("Giocatori anomali (1.0 e unico gol nell'ultima giornata):", list(anomalie))


Giocatori con probabilità esattamente 1.0: []
Giocatori con probabilità esattamente 0.0: []
Giocatori che hanno segnato solo in G38: ['alex sandro', 'keinan davis', 'alessandro deiola', 'kingstone mutandwa', 'junior sambia', 'mattia viti']
Giocatori anomali (1.0 e unico gol nell'ultima giornata): []


“Dopo l’applicazione del Laplace smoothing, si è osservato un aumento eccessivo delle probabilità per giocatori con un’unica osservazione, come Alex Sandro. Per mitigare questo effetto, è stato introdotto un peso correttivo in base al numero reale di transizioni osservate dallo stato attuale.”

In [6]:
import pandas as pd
import numpy as np

# Caricamento file
df = pd.read_csv('GolGiocatoriTemporale.csv')

# Estrai dinamicamente le colonne G1 ... Gn
giornate = [col for col in df.columns if col.startswith('G') and col[1:].isdigit()]
nome_colonna_ultima = giornate[-1]
print(f"Ultima giornata trovata: {nome_colonna_ultima}")
print(f"Colonne giornate trovate: {giornate}")

nomi = df['Giocatore']
dati = df[giornate].copy()

# Funzione: da gol a stato discreto
def gol_to_stato(x):
    if x == 0:
        return 0
    elif x == 1:
        return 1
    else:
        return 2

dati_stati = dati.map(gol_to_stato)

# Calcolo probabilità previsione con Laplace smoothing
prob_previste = []

for idx, row in dati_stati.iterrows():
    transizioni = np.ones((3, 3))  # Laplace smoothing: inizializza con 1 in ogni cella

    for i in range(len(row) - 1):  # da G1 a G(n-1)
        stato_corrente = row.iloc[i]
        stato_successivo = row.iloc[i + 1]
        transizioni[stato_corrente][stato_successivo] += 1

    # Calcolo matrice di probabilità
    somma_per_riga = transizioni.sum(axis=1, keepdims=True)
    matrice_prob = transizioni / somma_per_riga

    # Stato attuale (Gn)
    stato_attuale = row.iloc[-1]
    prob = matrice_prob[stato_attuale][1] + matrice_prob[stato_attuale][2]

    # Penalizza chi ha meno di 5 transizioni dallo stato attuale
    trans_count = somma_per_riga[stato_attuale][0]  # numero reale di transizioni
    if trans_count < 5:
        weight = trans_count / 5  # es. 2 → 0.4
        prob *= weight

    prob_previste.append(prob)

# Nome colonna dinamico (es. Prob_Gol_39_Markov)
col_markov = f'Prob_Gol_{len(giornate)+1}'

df_markov = pd.DataFrame({
    'Giocatore': nomi,
    col_markov: prob_previste
})

# Visualizza le prime righe
df_markov.head()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_markov)


Ultima giornata trovata: G38
Colonne giornate trovate: ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G27', 'G28', 'G29', 'G30', 'G31', 'G32', 'G33', 'G34', 'G35', 'G36', 'G37', 'G38']


,Giocatore,Prob_Gol_39
0,federico gatti,0.138889
1,alessandro bastoni,0.076923
2,andrea carboni,0.076923
3,tammy abraham,0.076923
4,francesco acerbi,0.135135
5,yacine adli,0.076923
6,alex sandro,0.400000
7,pontus almqvist,0.078947
8,amir rrahmani,0.108108
9,houssem aouar,0.333333


Il modello Markov ragiona solo sullo stato attuale e sulle transizioni passate da quello stato, non tiene conto del numero complessivo di gol, né della "qualità" del giocatore

Soluzione più elegante: probabilità pesata per "gol totali"


In [7]:
import pandas as pd
import numpy as np

# Carica il file
df = pd.read_csv('GolGiocatoriTemporale.csv')

# Estrai le colonne delle giornate dinamicamente
giornate = [col for col in df.columns if col.startswith('G') and col[1:].isdigit()]
nome_colonna_ultima = giornate[-1]
print(f"Ultima giornata trovata: {nome_colonna_ultima}")
print(f"Colonne giornate trovate: {giornate}")

nomi = df['Giocatore']
dati = df[giornate].copy()

# Funzione per mappare i gol in stati
def gol_to_stato(x):
    if x == 0:
        return 0
    elif x == 1:
        return 1
    else:
        return 2

# Converti i valori in stati (0, 1, 2)
dati_stati = dati.map(gol_to_stato)

# Lista per raccogliere le probabilità finali
prob_previste = []

# Ciclo su ogni giocatore
for idx, row in dati_stati.iterrows():
    transizioni = np.ones((3, 3))  # Laplace smoothing: inizializza a 1

    for i in range(len(row) - 1):
        stato_corrente = row.iloc[i]
        stato_successivo = row.iloc[i + 1]
        transizioni[stato_corrente][stato_successivo] += 1

    # Calcolo della matrice di probabilità
    somma_per_riga = transizioni.sum(axis=1, keepdims=True)
    matrice_prob = transizioni / somma_per_riga

    stato_attuale = row.iloc[-1]
    prob = matrice_prob[stato_attuale][1] + matrice_prob[stato_attuale][2]

    # Penalizza chi ha pochi dati
    trans_count = somma_per_riga[stato_attuale][0]
    if trans_count < 5:
        weight_trans = trans_count / 5
        prob *= weight_trans

    # Gol totali nel dataset
    gol_totali = df.loc[idx, giornate].sum()
    peso_gol = np.log1p(gol_totali) / 2.5  # normalizzato

    # Applica peso finale in base ai gol
    prob *= peso_gol

    # Clip per evitare valori eccessivi
    prob = min(prob, 1.0)

    prob_previste.append(prob)

# Colonna dinamica
col_markov = f'Prob_Gol_{len(giornate)+1}'

# Crea il DataFrame dei risultati
df_markov = pd.DataFrame({
    'Giocatore': nomi,
    col_markov: prob_previste
})

# Mostra le prime righe
df_markov.head()

# Ricalcola i gol totali (se non già presenti)
df_markov['Gol_Stagionali'] = df[giornate].sum(axis=1)

# Ordina per probabilità decrescente
df_risultati = df_markov.sort_values(by=col_markov, ascending=False).reset_index(drop=True)

# Mostra tutte le righe (oppure metti .head(50) per i top 50)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_risultati)

#  Salva solo le colonne 'Giocatore' e la probabilità Markov
df_risultati[[ 'Giocatore', col_markov ]].to_csv("Prob_Gol_Markov.csv", index=False)
print(" File salvato come Prob_Gol_Markov.csv (solo colonne selezionate)")

# 2. Scarica il file in locale da Colab
from google.colab import files
files.download("Prob_Gol_Markov.csv")


Ultima giornata trovata: G38
Colonne giornate trovate: ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G27', 'G28', 'G29', 'G30', 'G31', 'G32', 'G33', 'G34', 'G35', 'G36', 'G37', 'G38']


,Giocatore,Prob_Gol_39,Gol_Stagionali
0,olivier giroud,0.679971,16
1,gianluca scamacca,0.512990,12
2,lautaro martinez,0.503824,24
3,rafael leao,0.502382,9
4,m'baye niang,0.486478,6
5,romelu lukaku,0.439706,12
6,federico chiesa,0.418652,9
7,duvan zapata,0.400409,13
8,albert gudmundsson,0.390971,13
9,nicolas gonzalez,0.382293,11


 File salvato come Prob_Gol_Markov.csv (solo colonne selezionate)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
#  Calcolo della matrice di transizione per "lautaro martinez"
nome_giocatore = "lautaro martinez"

# Trova l'indice del giocatore
idx_lautaro = df[df['Giocatore'].str.lower() == nome_giocatore.lower()].index[0]

# Estrai la sua sequenza di stati e di gol
row = dati_stati.iloc[idx_lautaro]
row_gol = df.loc[idx_lautaro, giornate]

# Inizializza matrice con Laplace smoothing (1)
transizioni = np.ones((3, 3))

# Popola le transizioni
for i in range(len(row) - 1):
    stato_corrente = row.iloc[i]
    stato_successivo = row.iloc[i + 1]
    transizioni[stato_corrente][stato_successivo] += 1

# Calcola la matrice di probabilità
somma_per_riga = transizioni.sum(axis=1, keepdims=True)
matrice_prob = transizioni / somma_per_riga

# Stato attuale = ultima giornata
stato_attuale = row.iloc[-1]
prob_pura = matrice_prob[stato_attuale][1] + matrice_prob[stato_attuale][2]

# Gol totali nella stagione
gol_totali = row_gol.sum()
peso_gol = np.log1p(gol_totali) / 2.5
prob_finale = min(prob_pura * peso_gol, 1.0)

# Etichette leggibili
etichetta_stato = ['0 gol', '1 gol', '2+ gol']
df_transizioni = pd.DataFrame(transizioni, columns=etichetta_stato, index=etichetta_stato).astype(int)
df_prob = pd.DataFrame(matrice_prob, columns=etichetta_stato, index=etichetta_stato).round(3)

# Stampa dettagliata del procedimento
print(f"\n Giocatore analizzato: {nome_giocatore.title()}")
print(f"Stato attuale (G{len(giornate)}): {stato_attuale} → {etichetta_stato[stato_attuale]}")
print(f"Probabilità pura (Markov): {prob_pura:.6f}")
print(f"Gol totali stagionali: {int(gol_totali)}")
print(f"Peso log(1+gol)/2.5: {peso_gol:.6f}")
print(f"Probabilità finale (pura × peso): {prob_finale:.6f}")

# Matrici leggibili
print("\nMatrice di transizione (frequenze grezze con Laplace):")
display(df_transizioni)

print("\nMatrice di probabilità (normalizzata):")
display(df_prob)



 Giocatore analizzato: Lautaro Martinez
Stato attuale (G38): 0 → 0 gol
Probabilità pura (Markov): 0.391304
Gol totali stagionali: 24
Peso log(1+gol)/2.5: 1.287550
Probabilità finale (pura × peso): 0.503824

Matrice di transizione (frequenze grezze con Laplace):


,0 gol,1 gol,2+ gol
0 gol,14,7,2
1 gol,7,4,4
2+ gol,3,4,1



Matrice di probabilità (normalizzata):


,0 gol,1 gol,2+ gol
0 gol,0.609,0.304,0.087
1 gol,0.467,0.267,0.267
2+ gol,0.375,0.500,0.125
